In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from PIL import Image
import face_recognition
import pickle
import cv2
from pylab import *
from scipy.spatial.distance import cosine
import imgaug.augmenters as iaa
import os
from sklearn.preprocessing import normalize

#### Initialize paths of folders and threshold value of Cosine

In [ ]:
# Folder of event pictures
dossier_event = r"C:\Users\kador\photos-private"

# Folder of identity pictures (unique)
dossier_identity = r"C:\Users\kador\Downloads\ML\photos-perso"

# Initialize the threshold of Cosin Coefficient
threshold = 0.95

# 1) Define our functions

### a) Functions for loading the images, detecting faces, using MTCNN, extracting vectors

In [ ]:
# Load our image
def load_image(file_path):
    return face_recognition.load_image_file(file_path)

# Detect faces from the loaded image
def detect_faces(image):
    return MTCNN().detect_faces(image)

# Calculate cosine similarity and return the result
def calculate_cosine_similarity(vector1, vector2):
    vector1 = normalize([vector1])[0]
    vector2 = normalize([vector2])[0]
    return 1 - cosine(vector1, vector2)

# Detect faces with MTCNN and extract a list of them
def detect_faces_and_extract(photo_path):
    photo_load = face_recognition.load_image_file(photo_path)
    faces = MTCNN().detect_faces(photo_load)
    faces_list = []
    # For each face we extract the coordinates of its location
    for i, face in enumerate(faces):
        x, y, width, height = face['box']
        face_image = photo_load[y:y+height, x:x+width]
        faces_list.append(face_image)

    return photo_load, faces_list

# Extract the face vector
def extract_face_vector_event(face):
    face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(face_rgb)
    if face_locations:
      # The face vector is obtained with the first element of the function face_encodings()
        face_vector = face_recognition.face_encodings(face_rgb, face_locations)[0]
        return face_vector
    else:
        return None

# Extract the face vector of the identity WITHOUT using data augmentation
def extract_face_vector_identity_no_aug(photo_p_load):
    # The face vector is obtained with the first element of the function face_encodings()
    face_vector_small = face_recognition.face_encodings(photo_p_load)[0]
    return face_vector_small

### b) Function to extract face vector using data augmentation method

In [ ]:
# Extract the face vector of the identity by USING DATA AUGMENTATION
def extract_face_vector_identity_aug(photo_p_load):

    # Définir un séquenceur d'augmentation d'images
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.GaussianBlur((0, 3.0)),
        iaa.Affine(rotate=(-45, 45)),
        iaa.AdditiveGaussianNoise(scale=(0, 0.1*255))
    ], random_order=True)

    # Create a list to store augmented images
    augmented_images = [photo_p_load]

    # Apply augmentation and add augmentation version to the list
    for _ in range(8):
        augmented_images.append(seq(images=[photo_p_load])[0])

    # Create a list to store all face vectors
    general_vector = []

    # Create an MTCNN detector
    detector = MTCNN()

    # Process each augmented image
    for idx, img in enumerate(augmented_images):

        # Detect faces using MTCNN
        faces = detector.detect_faces(img)

        # Display only the extracted faces
        for face in faces:
            x, y, width, height = face['box']

            # If face is found, extract face vector
            face_location = [(y, x + width, y + height, x)]

            # Extracting only the face detected with the first element of the function face_encodings
            face_vector_small = face_recognition.face_encodings(img, face_location)[0]
            general_vector.append(face_vector_small)

    # Return the general vector containing all face vectors
    # Initialize the weighted average vector
    weighted_average_vector = [0] * len(general_vector[0])

    # Define weights for each vector
    weights = [0.5] + [0.5 / (len(general_vector) - 1)] * (len(general_vector) - 1)

    # Calculate the weighted sum
    for weight, vector in zip(weights, general_vector):
        weighted_average_vector = [w + weight * v for w, v in zip(weighted_average_vector, vector)]

    # Normalize the result
    total_weight = sum(weights)
    weighted_average_vector = [v / total_weight for v in weighted_average_vector]
    return weighted_average_vector

# 2) Predict identities with data augmentation

### a) Extract faces' vectors on identity pictures (with data augmentation)

In [ ]:
# Initialize a list to store results
results_dict = {}

# Loop through all files in the directory
for filename in os.listdir(dossier_identity):
    # Construct the full path to the image
    image_path = os.path.join(dossier_identity, filename)
    # Load the image
    IMAGE = load_image(image_path)
    # Compute the vectors of faces WITH data augmentation (see the function above)
    result = extract_face_vector_identity_aug(IMAGE)
    # Append the result to the list along with the file name
    results_dict[filename] = result

### b) Loop on every event pictures (explained in our report with a figure)

In [ ]:
# Association of EVENT and IDENTITY photo initialisation
list_association = {}

# START OF THE BIG LOOP #
##################################
# For each photo of the EVENT... #
##################################
for index_event, photo_event in enumerate(os.listdir(dossier_event)):

    photo_event_path = f"{dossier_event}/{photo_event}"
    photo_event_load, faces_list_event = detect_faces_and_extract(photo_event_path)

    ##########################################
    # For each FACE detected in the photo...#
    ##########################################
    for index_face, face in enumerate(faces_list_event):
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

        # Extract face locations from the image
        face_locations = face_recognition.face_locations(face_rgb)

        # Check if at least one face is detected
        if face_locations:
            # Extract face encodings (vectors) for the first face
            vector_event = face_recognition.face_encodings(face_rgb, face_locations)[0]

        list_cosine = []
        index_to_photo_identity = {}

        ################################
        # For each unique IDENTITY ... #
        ################################
        i=0
        for photo_identity, vector_identity in results_dict.items():
            index_to_photo_identity[i] = photo_identity

            # Compute the cosine similarity between the two vectors
            cosine_similarity =  calculate_cosine_similarity(vector_identity, vector_event)
            list_cosine.append(cosine_similarity if cosine_similarity >= threshold else 0)
            i=i+1
        print(f"\n>>Face n°{index_face} of photo n°{index_event} done")
        # Only keep the highest cosine coefficient
        max_cosine = max(list_cosine)
        if max_cosine != 0 :
            identity_associated = index_to_photo_identity.get(list_cosine.index(max_cosine))

            # Associate the identity to the event photo
            list_association[(photo_event, index_face)] = (identity_associated, index_face)
            print(f'\n>>>Association found with max cosine similarity of {round(max_cosine, 8)}!\nPhoto {photo_event}_N°{index_face} associated to identity {identity_associated}')

print(f'\nSummary of association:\n{list_association}')
# END OF THE BIG LOOP #

### c) Display all photos' names where the identity is detected

In [ ]:
# Initialize the list
data = []
photo_identity0 = ""
photo_event0 = ""

# For any identity detected on event photos
for photo_event, photo_identity in list_association.items():
    photo_identity0 = photo_identity[0]
    photo_event0 = photo_event[0]

    # Add it to our list
    data.append(f"{photo_identity0} {photo_event0}")

# Clean extension names
names_list = [item.split("g ")[0] for item in data]
unique_names = set(names_list)
def clean_extension(name):
    if name.endswith(".jp"):
        return name + "g"
    elif name.endswith(".jpe"):
        return name + "g"
    else:
        return name

# For every identity
unique_cleaned_names = set(clean_extension(name) for name in unique_names)
for item in data:
    # Use the correct index for item
    name = item.split("g ")[0]
    name=name+"g"
    # Check if the name is in the unique_cleaned_names set
    if name in unique_cleaned_names:
        link = item.split("g ")[1]
        # Check if the name already exists in the dictionary
        if name not in result_dict:
            # If it doesn't exist, create a new entry with the link as a list
            result_dict[name] = [link]
        else:
            # If it already exists, append the link to the existing list
            result_dict[name].append(link)

# Our result is a dictionnary of key = identity and value = the photos where the identity was detected
# This output is the result we need to implement our product onto our app (participants want to get only
# the pictures where they appear)
unique_result = {key: list(set(value)) for key, value in result_dict.items()}
print(unique_result)

### d) Display the results on the event pictures
##### NB : we decided to extract each unique identity to the event picture, so that it is sent uniquely to the client

In [ ]:
for photo_event, photo_identity in list_association.items():
    big_photo_path = f'{dossier_event}/{photo_event[0]}'
    big_photo = face_recognition.load_image_file(big_photo_path)
    # Detect faces using MTCNN
    big_faces = MTCNN().detect_faces(big_photo)
    # Plot the big photo with rectangles around faces
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(big_photo)

    # Extracted faces from the big photo
    big_face_images = []

    for i, face in enumerate(big_faces):

        x, y, width, height = face['box']

        # Extract and store the face image
        face_image = big_photo[y:y+height, x:x+width]
        big_face_images.append(face_image)

        # Draw rectangle around the face
        rect = Rectangle((x, y), width, height, fill=False, color='red')
        ax.add_patch(rect)

        if i == photo_event[1]:
            # Draw rectangle around the face
            rect = Rectangle((x, y), width, height, fill=False, color='blue')
            ax.add_patch(rect)
            # Customize appearance of the number
            number_text = ax.text(x, y, photo_identity[0], color='blue', fontsize=8, weight='bold')
        else:
            number_text = ax.text(x, y, " ", color='red', fontsize=8, weight='bold')
            # Draw rectangle around the face
            rect = Rectangle((x, y), width, height, fill=False, color='red')
            ax.add_patch(rect)

    # Display the plot
    plt.show()


# 3) Predict identities without data augmentation (testing phase)

### a) Extract faces' vectors on identity pictures (without data augmentation)

In [ ]:
vectors_no_aug = {}
# Loop through all files in the directory
for filename in os.listdir(dossier_identity):
    # Construct the full path to the image
    image_path = os.path.join(dossier_identity, filename)

    # Load the image
    IMAGE = load_image(image_path)
    # Compute faces vectors WITHOUT data augmentation (see function above)
    result = extract_face_vector_identity_no_aug(IMAGE)

    # Append the result to the list along with the file name
    vectors_no_aug[filename] = result

### b) Loop on every event pictures


In [ ]:
# Association of EVENT and IDENTITY photo initialisation
list_association = {}

# START OF THE BIG LOOP #
##################################
# For each photo of the EVENT... #
##################################
for index_event, photo_event in enumerate(os.listdir(dossier_event)):

    photo_event_path123 = f"{dossier_event}/{photo_event}"
    photo_event_load, faces_list_event = detect_faces_and_extract(photo_event_path123)

    ##########################################
    # For each FACE detected in the photo...#
    ##########################################
    for index_face, face in enumerate(faces_list_event):
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

        # Extract face locations from the image
        face_locations = face_recognition.face_locations(face_rgb)

        # Check if at least one face is detected
        if face_locations:
            # Extract face encodings (vectors) for the first face
            vector_event = face_recognition.face_encodings(face_rgb, face_locations)[0]

        list_cosine = []
        index_to_photo_identity = {}

        ################################
        # For each unique IDENTITY ... #
        ################################
        i=0
        for photo_identity, vector_identity in vectors_no_aug.items():
            index_to_photo_identity[i] = photo_identity

            # Compute the cosine similarity between the two vectors
            cosine_similarity =  calculate_cosine_similarity(vector_identity, vector_event)
            list_cosine.append(cosine_similarity if cosine_similarity >= threshold else 0)
            i=i+1
        print(f"\n>>Face n°{index_face} of photo n°{index_event} done")
        # Only keep the highest cosine coefficient
        max_cosine = max(list_cosine)
        if max_cosine != 0 :
            identity_associated = index_to_photo_identity.get(list_cosine.index(max_cosine))

            # Associate the identity to the event photo
            list_association[(photo_event, index_face)] = (identity_associated, index_face)
            print(f'\n>>>Association found with max cosine similarity of {round(max_cosine, 8)}!\nPhoto {photo_event}_N°{index_face} associated to identity {identity_associated}')

print(f'\nSummary of association:\n{list_association}')
# END OF THE BIG LOOP #

for photo_event, photo_identity in list_association.items():
    print(photo_event, photo_identity)

### c) Display the results on the event pictures
##### NB : we decided to extract each unique identity to the event picture, so that it is sent uniquely to the client

In [ ]:
for photo_event, photo_identity in list_association.items():
    big_photo_path = f'{dossier_event}/{photo_event[0]}'
    big_photo = face_recognition.load_image_file(big_photo_path)
    # Detect faces using MTCNN
    big_faces = MTCNN().detect_faces(big_photo)
    # Plot the big photo with rectangles around faces
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(big_photo)

    # Extracted faces from the big photo
    big_face_images = []

    for i, face in enumerate(big_faces):

        x, y, width, height = face['box']

        # Extract and store the face image
        face_image = big_photo[y:y+height, x:x+width]
        big_face_images.append(face_image)

        # Draw rectangle around the face
        rect = Rectangle((x, y), width, height, fill=False, color='red')
        ax.add_patch(rect)

        if i == photo_event[1]:
            # Draw rectangle around the face
            rect = Rectangle((x, y), width, height, fill=False, color='blue')
            ax.add_patch(rect)
            # Customize appearance of the number
            number_text = ax.text(x, y, photo_identity[0], color='blue', fontsize=8, weight='bold')
        else:
            number_text = ax.text(x, y, " ", color='red', fontsize=8, weight='bold')
            # Draw rectangle around the face
            rect = Rectangle((x, y), width, height, fill=False, color='red')
            ax.add_patch(rect)

    # Display the plot
    plt.show()